![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_AlbertForQuestionAnswering.ipynb)

# Import OpenVINO AlbertForQuestionAnswering models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting AlbertForQuestionAnswering models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for AlbertForQuestionAnswering from AlbertForQuestionAnswering and they have to be in `Question Answering` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.6/342.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [twmkn9/albert-base-v2-squad2](https://huggingface.co/twmkn9/albert-base-v2-squad2) model from HuggingFace as an example and load it as a `OVModelForQuestionAnswering`, representing an OpenVINO model.
- In addition to the AlbertForQuestionAnswering model, we also need to save the `AlbertTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [ ]:
from transformers import AutoTokenizer
from optimum.intel import OVModelForQuestionAnswering

MODEL_NAME = "twmkn9/albert-base-v2-squad2"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ort_model = OVModelForQuestionAnswering.from_pretrained(MODEL_NAME, export=True)
ort_model.save_pretrained(EXPORT_PATH)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at twmkn9/albert-base-v2-squad2 were not used when initializing AlbertForQuestionAnswering: ['albert.pooler.bias', 'albert.pooler.weight']
- This IS expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.7M [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('onnx_models/twmkn9/albert-base-v2-squad2/tokenizer_config.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/special_tokens_map.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/spiece.model',
 'onnx_models/twmkn9/albert-base-v2-squad2/added_tokens.json',
 'onnx_models/twmkn9/albert-base-v2-squad2/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 46676
-rw-r--r-- 1 root root      844 Jun 18 00:51 config.json
-rw-r--r-- 1 root root 44382400 Jun 18 00:51 openvino_model.bin
-rw-r--r-- 1 root root   360305 Jun 18 00:51 openvino_model.xml
-rw-r--r-- 1 root root      286 Jun 18 00:51 special_tokens_map.json
-rw-r--r-- 1 root root   760289 Jun 18 00:51 spiece.model
-rw-r--r-- 1 root root     2616 Jun 18 00:51 tokenizer_config.json
-rw-r--r-- 1 root root  2275547 Jun 18 00:51 tokenizer.json


- As you can see, we need to move `spiece.model` from the tokenizer to `assets` folder which Spark NLP will look for

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/spiece.model  {EXPORT_PATH}/assets

In [5]:
!ls -l {EXPORT_PATH}/assets

total 744
-rw-r--r-- 1 root root 760289 Jun 18 00:51 spiece.model


Voila! We have our `spiece.model` inside assets directory

## Import and Save AlbertForQuestionAnswering in Spark NLP


- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 32.6 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `AlbertForQuestionAnswering` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `AlbertForQuestionAnswering` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.



In [8]:
from sparknlp.annotator import AlbertForQuestionAnswering

spanClassifier = AlbertForQuestionAnswering.loadSavedModel(
     f"{EXPORT_PATH}",
     spark
 )\
  .setInputCols(["document_question",'document_context'])\
  .setOutputCol("answer")\
  .setCaseSensitive(False)\
  .setMaxSentenceLength(512)

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
spanClassifier.write().overwrite().save("./{}_spark_nlp_openvino".format(MODEL_NAME))

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome 😎  !

This is your AlbertForQuestionAnswering model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 44452
-rw-r--r-- 1 root root 44749813 Jun 18 00:53 albert_classification_openvino
-rw-r--r-- 1 root root   760289 Jun 18 00:53 albert_spp
drwxr-xr-x 2 root root     4096 Jun 18 00:53 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny AlbertForQuestionAnswering model in Spark NLP 🚀 pipeline!

In [12]:
from sparknlp.base import MultiDocumentAssembler
from sparknlp.annotator import AlbertForQuestionAnswering
from pyspark.ml import Pipeline

document_assembler = MultiDocumentAssembler()\
    .setInputCols(["question", "context"])\
    .setOutputCols(["document_question", "document_context"])

spanClassifier_loaded = AlbertForQuestionAnswering.load(f"./{MODEL_NAME}_spark_nlp_openvino")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")

pipeline = Pipeline().setStages([
    document_assembler,
    spanClassifier_loaded
])

example = spark.createDataFrame([[
    "What's my name?",
    "My name is Clara and I live in Berkeley."
]]).toDF("question", "context")

result = pipeline.fit(example).transform(example)
result.select("answer.result").show(1, False)

+-------+
|result |
+-------+
|[clara]|
+-------+



That's it! You can now go wild and use hundreds of `AlbertForQuestionAnswering` models from HuggingFace 🤗 in Spark NLP 🚀
